Install d3rlpy and d4rl-atari.

In [ ]:
!pip install git+https://github.com/takuseno/d4rl-atari d3rlpy

Setup dataset.

In [ ]:
import d3rlpy

# prepare 1% Breakout dataset
dataset, env = d3rlpy.datasets.get_atari_transitions(
    "breakout",
    fraction=0.01,
    index=0,
    num_stack=4,
)

Setup data-driven deep reinforcement learning.

In [ ]:
cql = d3rlpy.algos.DiscreteCQLConfig(
    learning_rate=5e-5,
    optim_factory=d3rlpy.optimizers.AdamFactory(eps=1e-2 / 32),
    batch_size=32,
    alpha=4.0,
    q_func_factory=d3rlpy.models.q_functions.QRQFunctionFactory(
        n_quantiles=200
    ),
    observation_scaler=d3rlpy.preprocessing.PixelObservationScaler(),
    target_update_interval=2000,
    reward_scaler=d3rlpy.preprocessing.ClipRewardScaler(-1.0, 1.0),
).create(device="cuda:0")

env_scorer = d3rlpy.metrics.EnvironmentEvaluator(env, epsilon=0.001)

cql.fit(
    dataset,
    n_steps=1000000,
    n_steps_per_epoch=10000,
    evaluators={"environment": env_scorer},
)